# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
total_q=data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()


total_q

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [8]:
pivoted_total=pd.pivot_table(total_q, index='ProductName',columns='CustomerID',values="Quantity").fillna(0)
pivoted_total.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivoted_total.T, 'euclidean'))), 
                         index=pivoted_total.columns, columns=pivoted_total.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [11]:
similar=list(distances[33].sort_values(ascending=False)[1:6].index)
similar

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
similar_df=total_q[total_q['CustomerID'].isin(similar)]

similar_df

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [19]:
agg_df=similar_df.groupby(['ProductName'])['Quantity'].sum().reset_index()


agg_df=agg_df.sort_values(['Quantity'],ascending=False)

agg_df

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3
...,...,...
85,Hinge W Undercut,1
86,Ice Cream Bar - Hageen Daz To,1
87,Jagermeister,1
88,Jolt Cola - Electric Blue,1


In [ ]:
agg_df.reset_index(inplace=True)

agg_df.columns

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [31]:
items_customer=pivoted_total[33].to_frame()

recommended=pd.merge(agg_df,items_customer, on='ProductName')

recommended=recommended[recommended[33]==0]

recommended

,ProductName,Quantity,33
0,Butter - Unsalted,3,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0
3,Soup - Campbells Bean Medley,3,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0
6,Chicken - Soup Base,2,0.0
...,...,...,...
211,Halibut - Steaks,1,0.0
213,Hinge W Undercut,1,0.0
214,Ice Cream Bar - Hageen Daz To,1,0.0
215,Jagermeister,1,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [33]:
recomendations=dict()
index_lst=list(distances.index)
for e in index_lst:
    similar=list(distances[e].sort_values(ascending=False)[1:6].index)
    data1=total_q[total_q["CustomerID"].isin(similar)]
    data2=data1.groupby(by=["ProductName"]).agg({"Quantity": "sum"}).sort_values(by="Quantity",ascending=False).reset_index()
    index_lst = list(data[data['CustomerID'] == e]['ProductName'])
    data3=data2[data2["ProductName"].isin(index_lst)].head()
    recomendations[e]=list(data3["ProductName"])
recomendations[33]

['Towels - Paper / Kraft',
 'Pepper - Black, Whole',
 'Potatoes - Idaho 100 Count',
 'Veal - Inside, Choice',
 'Lamb - Pieces, Diced']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [34]:
recomendations_df=pd.DataFrame(recomendations)

recomendations_df

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Towels - Paper / Kraft,"Pepper - Black, Whole",Potatoes - Idaho 100 Count,Wine - Ej Gallo Sierra Valley,"Veal - Inside, Choice",Cheese - Cambozola,Soup - Campbells Bean Medley,Halibut - Steaks,Ketchup - Tomato,"Sauce - Gravy, Au Jus, Mix",...,Pork - Kidney,Ecolab - Lime - A - Way 4/4 L,"Wine - Red, Colio Cabernet","Thyme - Lemon, Fresh",Scampi Tail,Bouq All Italian - Primerba,Chocolate - Dark,Chips Potato All Dressed - 43g,Halibut - Steaks,Sea Bass - Whole
1,"Pepper - Black, Whole",Lamb - Ground,Wine - Blue Nun Qualitatswein,Wine - Blue Nun Qualitatswein,Soupfoamcont12oz 112con,"Nut - Chestnuts, Whole","Sauce - Gravy, Au Jus, Mix",Fenngreek Seed,Fenngreek Seed,Cheese - Cambozola,...,Cocoa Butter,Cocoa Butter,Dc Hikiage Hira Huba,Table Cloth 54x72 White,Beef - Texas Style Burger,Tea - Jasmin Green,Longos - Grilled Salmon With Bbq,Table Cloth 81x81 White,Cod - Black Whole Fillet,Apricots - Halves
2,Potatoes - Idaho 100 Count,Coffee - Irish Cream,Lettuce - Spring Mix,Table Cloth 81x81 White,Curry Paste - Madras,Ketchup - Tomato,Chinese Foods - Chicken,Sardines,Halibut - Steaks,Soupfoamcont12oz 112con,...,Mustard - Seed,Vinegar - Tarragon,Clam Nectar,Barramundi,French Pastry - Mini Chocolate,Arizona - Green Tea,Olives - Kalamata,Pernod,Wine - Pinot Noir Latour,Onion Powder
3,"Veal - Inside, Choice","Tart Shells - Sweet, 4",Cheese Cloth No 100,Potatoes - Idaho 100 Count,"Fish - Scallops, Cold Smoked",Cheese Cloth No 100,Sausage - Breakfast,"Pepper - Paprika, Hungarian",Soupfoamcont12oz 112con,Halibut - Steaks,...,"Hickory Smoke, Liquid","Veal - Brisket, Provimi,bnls",Cornflakes,Cake - Cake Sheet Macaroon,Scallops - Live In Shell,Cinnamon Buns Sticky,Sugar - Fine,"Ice - Clear, 300 Lb For Carving",Chicken - Wieners,Dried Figs
4,"Lamb - Pieces, Diced",Tahini Paste,Towels - Paper / Kraft,Corn Meal,Foam Dinner Plate,Tea - Decaf Lipton,Crush - Cream Soda,"Oregano - Dry, Rubbed","Mushrooms - Black, Dried",Ocean Spray - Ruby Red,...,"Thyme - Lemon, Fresh","Appetizer - Mini Egg Roll, Shrimp",Coffee - Dark Roast,Longos - Chicken Wings,Chinese Foods - Chicken,Bacardi Breezer - Tropical,Mustard Prepared,"Wine - White, Mosel Gold",Turkey - Oven Roast Breast,Chips Potato Salt Vinegar 43g


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.